In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from shutil import copyfile
from sklearn.model_selection import train_test_split
from IPython.display import SVG,display
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.applications import VGG16
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import load_img

/home/diegues/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# CNN Tuning
    - Try different LR (starting from 1.0 to 0.001) and see differences (no. of epochs, resulting accuracies and performance)
    - Compare with using Adam and Nesterov Momentum SGD
    - Try out different CNNs designs (DenseNet, ResNet50, Inception v3)
    - Compare performances and scores (no.epochs, accuracies, etc)
    - Add more samples
    - Oversampling

# Data Import

In [4]:
folder = '/media/newdrive/20180501/ProcessedImages/'
train_path = folder + 'SampledData/train/'
val_path = folder + 'SampledData/validation/'
test_path = folder + 'test/'

train_df = pd.read_csv(folder + 'sampled_data.csv').set_index('filename')
test_df = pd.read_csv(folder + 'unknown2-targets.csv').set_index('filename')

print(train_df.head())
print(test_df.head())

                                        timestamp  latitude  longitude   roll  \
filename                                                                        
113610_cam_survey_1_frame1246.jpg    1.525175e+09  41.53323   -8.81036  -3.11   
113610_cam_survey_1_frame1272.jpg    1.525175e+09  41.53324   -8.81036  -0.24   
132143_forcadinho-np3_frame1317.jpg  1.525183e+09  41.53305   -8.80962  -3.47   
113610_cam_survey_1_frame2894.jpg    1.525176e+09  41.53307   -8.80638   0.87   
113610_cam_survey_1_frame2643.jpg    1.525176e+09  41.53337   -8.80586 -23.16   

                                     pitch    entropy      date  depth  \
filename                                                                 
113610_cam_survey_1_frame1246.jpg   -20.12  21.806570  01/05/18   9.67   
113610_cam_survey_1_frame1272.jpg   -21.64  21.847648  01/05/18   8.34   
132143_forcadinho-np3_frame1317.jpg -22.16  21.671617  01/05/18   9.81   
113610_cam_survey_1_frame2894.jpg   -17.67  21.931659  01/05/1

## Data Generators

In [3]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=360,
      vertical_flip=True,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=1,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        val_path,
        target_size=(224, 224),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)

test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(224, 224),
        batch_size=1,
        class_mode=None,
        shuffle=False)

Found 457 images belonging to 5 classes.
Found 197 images belonging to 5 classes.
Found 751 images belonging to 1 classes.


# CallBacks


In [6]:
tensorboard = TensorBoard(log_dir='./Graph/vgg16_1', histogram_freq=0, write_graph=True, write_images=True)
best_checkpoint = ModelCheckpoint('./weights/vgg16_1_cp.h5', monitor='val_acc', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_acc', patience=10, verbose=1)
reducer = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=3, verbose=1, min_lr=1e-7)

# Optimizers
## SGD

In [7]:
sgd1 = SGD(lr=1)
sgd0_1 = SGD(lr=.1)
sgd0_01 = SGD(lr=.01)
sgd0_001 = SGD(lr=.001)

# VGG16

In [ ]:
vgg = VGG16(weights='imagenet')
fc1 = vgg.layers[-3]
fc2 = vgg.layers[-2]
predictions = Dense(5, activation='softmax', name='predictions')


dropout1 = Dropout(0.5)
dropout2 = Dropout(0.5)

x = dropout1(fc1.output)
x = fc2(x)
x = dropout2(x)
predictors = predictions(x)

eunis_classifier_w_dropout = Model(input=vgg.input, output=predictors)
eunis_classifier_w_dropout.compile(loss='categorical_crossentropy',
                        optimizer=sgd1, 
                        metrics=['acc'])

# Training 


In [ ]:
history_w_drop = eunis_classifier_w_dropout.fit_generator(train_generator, 
                                         steps_per_epoch=457,
                                         epochs=500,
                                         validation_data=validation_generator,
                                         validation_steps = 197,
                                         callbacks=[tensorboard, best_checkpoint, earlystop, reducer]
                                        )